In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from glob import glob
import os

In [ ]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
def lr_schedule(epoch):
    if epoch <= 50:
        return 0.0001
    else: 
        return 0.00005

# Data Preparation

In [ ]:
%cd /home/caitsithx/courses/deeplearning1/nbs
DATA_DIR = '/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring'
TRAIN_DIR = DATA_DIR + '/train'
RESULT_DIR = DATA_DIR + '/result'

TRAIN_FEAT = RESULT_DIR + '/train_feats.dat'
VAL_FEAT = RESULT_DIR + '/val_feats.dat'

In [ ]:
df_train = pd.read_csv(DATA_DIR+'/train_labels.csv')
master = df_train
master.head()

In [ ]:
y = []
file_paths = []
for i in range(len(master)):
    file_paths.append( TRAIN_DIR + '/' + str(master.ix[i][0]) +'.jpg' )
    y.append(master.ix[i][1])
y = np.array(y)
print(y.shape)
print(len(file_paths))

In [ ]:
print(file_paths[:5])
print(y[:5])

In [ ]:
#image reseize & centering & crop 

def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    # centering
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized


x = []
for i, file_path in enumerate(file_paths):
    #read image
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))
    
    #out put 224*224px 
    img = img[16:240, 16:240]
    x.append(img)

x = np.array(x)
print(x.shape)

In [ ]:
x = np.transpose(x, (0, 3, 1, 2))

In [ ]:
sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")
img_path = DATA_DIR+"/test/unknown/"

test_names = []
file_paths = []

for i in range(len(sample_submission)):
    test_names.append(sample_submission.ix[i][0])
    file_paths.append( img_path + str(int(sample_submission.ix[i][0])) +'.jpg' )
    
test_names = np.array(test_names)
print(test_names.shape)

In [ ]:
file_paths[:5]

In [ ]:
test_images = []
for file_path in file_paths:
    #read image
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))
    
    #out put 224*224px 
    img = img[16:240, 16:240]
    test_images.append(img)
    
    path, ext = os.path.splitext( os.path.basename(file_paths[0]) )

test_images = np.array(test_images)

In [ ]:
print(test_images.shape)
test_images = np.transpose(test_images, (0, 3, 1, 2))
print(test_images.shape)

## Save image Data

In [ ]:
save_array(DATA_DIR+ '/train_imges.npy', x)
save_array(DATA_DIR+ '/test_imges.npy', test_images)

## split train and val

In [ ]:
x = load_array(DATA_DIR+ '/train_imges.npy')
test_images = load_array(DATA_DIR+ '/test_imges.npy')

In [ ]:
print(x.shape)
print(test_images.shape)

In [ ]:
data_num = len(y)
random_index = np.random.permutation(data_num)

x_shuffle = []
y_shuffle = []
for i in range(data_num):
    x_shuffle.append(x[random_index[i]])
    y_shuffle.append(y[random_index[i]])
    
x = np.array(x_shuffle) 
y = np.array(y_shuffle)

In [ ]:
val_split_num = int(round(0.2*len(y)))
x_train = x[val_split_num:]
y_train = y[val_split_num:]
x_test = x[:val_split_num]
y_test = y[:val_split_num]

print('x_train', x_train.shape)
print('y_train', y_train.shape)
print('x_test', x_test.shape)
print('y_test', y_test.shape)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Build Models

In [ ]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.optimizers import Adam
from keras.layers import Dropout, Flatten, Dense

img_rows, img_cols, img_channel = 224, 224, 3

In [ ]:
incep_model = applications.InceptionV3(include_top=False, 
                                       weights='imagenet', 
                                       input_shape=(img_rows, img_cols, img_channel))

In [ ]:
vgg16_model = applications.VGG16(weights='imagenet', 
                                 include_top=False, 
                                 input_shape=(img_rows, img_cols, img_channel))

In [ ]:
vgg19_model = applications.VGG19(include_top=False, 
                                 weights='imagenet', 
                                 input_shape=(img_rows, img_cols, img_channel))

In [ ]:
res_model = applications.ResNet50(
            include_top=False, 
            weights='imagenet',
            input_tensor=None, 
            input_shape=(img_rows, img_cols, img_channel))

In [ ]:
vgg19_model.summary()

In [ ]:
#base_model.output_shape[1:]
def get_dense_model(input_shape):
    dense_model = Sequential()
    dense_model.add(Flatten(input_shape=input_shape))
    
    dense_model.add(Dense(256, activation='relu'))
    dense_model.add(Dropout(0.25))
    dense_model.add(Dense(1, activation='sigmoid'))

    return dense_model

In [ ]:
models = []
models.append(Model(inputs=vgg16_model.input, 
                    outputs=get_dense_model(vgg16_model.output_shape[1:])(vgg16_model.output)))
models.append(Model(inputs=vgg19_model.input, 
                    outputs=get_dense_model(vgg19_model.output_shape[1:])(vgg19_model.output)))
models.append(Model(inputs=res_model.input, 
                    outputs=get_dense_model(res_model.output_shape[1:])(res_model.output)))
models.append(Model(inputs=incep_model.input, 
                    outputs=get_dense_model(incep_model.output_shape[1:])(incep_model.output)))

for model in models:
    model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

#model.summary()

# Train Models

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

batch_size=40
w_file = RESULT_DIR + '/xiaoliangl_id_{}.h5'

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)
vld_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
nb_epochs=10

for i, model in enumerate(models):
    print ("train model: %d " % i)
    model.fit_generator(
        train_datagen.flow(x_train, y_train, batch_size=batch_size),
        steps_per_epoch=x_train.shape[0] // batch_size,
        epochs=nb_epochs,
        callbacks=[ModelCheckpoint(w_file.format(i), 
                                   monitor='val_acc', 
                                   save_best_only=True),
                                   LearningRateScheduler(lr_schedule)],
        validation_data=(x_test, y_test),
        validation_steps=x_test.shape[0] // batch_size)

# Test

In [ ]:
test_images = test_images.astype('float32')
test_images /= 255

In [ ]:
preds = []
for i, model in enumerate(models):
    model.load_weights(w_file.format(i))
    pred = model.predict(test_images)
    preds.append(pred)
preds = np.mean(preds, axis=0)

In [ ]:
preds = models[1].predict(test_images)

In [ ]:
sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")

for i, name in enumerate(test_names):
    sample_submission.loc[sample_submission['name'] == name, 'invasive'] = preds[i]

sample_submission.to_csv(RESULT_DIR + "/submit2.csv", index=False)